In [1]:


import folium
from folium.plugins import HeatMap
import pandas as pd
import geopandas as gpd


In [2]:
df = pd.read_csv('C:\\Users\\ss6365\\Desktop\\11111\\Geolife\\Machine_Learning\\Laplace\\2km\\merged_averaged_laplace_0.1_2km.csv')
#df = pd.read_csv('/Users/shafizurrahmanseeam/Desktop/ARR/AR_GPS_Sensor_Data/Machine_Learning/Laplace/2km/merged_laplace_0.1_2km.csv')
#df = pd.read_csv('/Users/shafizurrahmanseeam/Desktop/ARR/AR_GPS_Sensor_Data/Perturbed/Laplace/merged_laplace_0.1.csv')

your_dataframe = df


In [3]:

# Sample DataFrame setup (replace this with your actual DataFrame)
# your_dataframe = pd.DataFrame({
#     'Latitude': [39.9042, 39.9132],
#     'Longitude': [116.4074, 116.4184]
# })

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(
    your_dataframe,
    geometry=gpd.points_from_xy(your_dataframe.Longitude, your_dataframe.Latitude))

# Determine the bounds of your data
bounds = gdf.total_bounds  # returns (minx, miny, maxx, maxy)
map_center = [(bounds[1] + bounds[3]) / 2, (bounds[0] + bounds[2]) / 2]

# Your Mapbox access token
mapbox_access_token = 'pk.eyJ1Ijoic2hhZml6dXJyYWhtYW5zZWVhbSIsImEiOiJjbHFhbWU0eGIxaDI0MmluenF1aDB4ZGtvIn0.wkGYw904a8nxHLH0k2G6jg'

# Use a predefined Mapbox style
mapbox_style = 'streets-v11'  # Example style

# Create the map with the Mapbox style
m = folium.Map(
    location=map_center,
    zoom_start=14,
    tiles='https://api.mapbox.com/styles/v1/mapbox/' + mapbox_style + '/tiles/{z}/{x}/{y}?access_token=' + mapbox_access_token,
    attr='Mapbox'
)

# Create a heatmap
heatmap_data = gdf[['Latitude', 'Longitude']].values.tolist()
heatmap = HeatMap(heatmap_data, min_opacity=0.4, radius=6, blur=3.5, max_zoom=1)
m.add_child(heatmap)

# Add a rectangle to represent the bounds
folium.Rectangle(
    bounds=[[bounds[1], bounds[0]], [bounds[3], bounds[2]]],
    color='#12a4d9',
    fill=True,
    fill_opacity=0.2,
).add_to(m)

# Define an outer box with an offset
offset = 0.0025  # Adjust as necessary for the desired spacing
outer_bounds = [[bounds[1] - offset, bounds[0] - offset], [bounds[3] + offset, bounds[2] + offset]]

# Create the dotted outer box
folium.PolyLine(
    locations=[outer_bounds[0], [outer_bounds[0][0], outer_bounds[1][1]], outer_bounds[1], [outer_bounds[1][0], outer_bounds[0][1]], outer_bounds[0]],
    color=' #d72631',
    weight=4,
    dash_array='10, 20',  # Controls the dash pattern: 5 pixels dash, 10 pixels space
).add_to(m)

# Display the map in the Jupyter Notebook
m

